# LeetCode 2281 Sum of Total Strength of Wizards

`def totalStrength(strength: List[int]) -> int`

1. For each `i` in `0 .. len(strength)-1`
   * Find first index `l` on the left side of `i` and first index `r` on the right side of `i` such that 
     * `strength[l] <= strength[i]`, `strength[r] < strength[i]`
     * `strength[i] < strength[l+1], ..., strength[i-1]`, `strength[i] <= strength[i+1], ..., strength[r-1]`
   * Can be calculated in one pass using monotonic stack
2. Preprocess `prefix` and `prefix_sum` arrays
   * `prefix[j] = strength[0] + strength[1] + ... + strength[j-1]`, and `prefix[0] = 0` 
   * `sum(strength[j..k]) = prefix[k+1] - prefix[j] = f(k+1) - f(j)`
   * `prefix_sum[j] = prefix[0] + prefix[1] + ... + prefix[j-1]`
3. For each `i` in `0 .. len(strength)-1`, the sum of strengths of contiguous groups is 
   $$
   \begin{aligned}
   \sum_{k=i}^{r-1}\sum_{j=l+1}^{i} \text{sum}(j..k) = & \sum_{k=i}^{r-1}\sum_{j=l+1}^{i} f(k+1) - f(j) \\
   = & \sum_{k=i}^{r-1} (i-l) f(k+1) - (f(l+1) + f(l+2) + \cdots + f(i)) \\
   = & (i - l) (f(i+1) + f(i+2) + \cdots + f(r)) - (r-i)(f(l+1) + f(l+2) + \cdots + f(i))
   \end{aligned}
   $$ 

$O(N)$ time and space complexity
  

In [1]:
from typing import List

In [2]:
def totalStrength(strength: List[int]) -> int:
    mod = 10**9 + 7
    n = len(strength)
    left = [-1] * n
    right = [n] * n
    # 1. calculate left and right bound
    stack = []
    for i in range(n):
        while stack and strength[stack[-1]] > strength[i]:
            j = stack.pop()
            right[j] = i
        stack.append(i)
    stack = []
    for i in range(n-1, -1, -1):
        while stack and strength[stack[-1]] >= strength[i]:
            j = stack.pop()
            left[j] = i
        stack.append(i)
    
    # 2. calculate prefix and prefix_sum
    prefix = [0] * (n+1)
    prefix_sum = [0] * (n+2)
    for i in range(n):
        prefix[i+1] = prefix[i] + strength[i]
        prefix_sum[i+1] = prefix_sum[i] + prefix[i]
    prefix_sum[n+1] = prefix_sum[n] + prefix[n]
    
    # 3. calculate total strengths of all contiguous groups
    res = 0
    for i in range(n):
        res += strength[i] * ((i - left[i]) * (prefix_sum[right[i] + 1] - prefix_sum[i+1]) % mod \
                                + 2 * mod \
                                - (right[i] - i) * (prefix_sum[i+1] - prefix_sum[left[i]+1]) % mod)
        res %= mod
    return res

In [3]:
# Test cases

inputs = [[1],
          [1,3,1,2],
          [1,3,1,2,4],
          [5,4,6,7,8,3,2,1,4,9]]
outputs = [1, 44, 100, 2326]

all_passed = True
for i in range(len(inputs)):
    ret = totalStrength(inputs[i])
    if ret != outputs[i]:
        print(f"Test case {i+1} failed: input = {inputs[i]}, expected = {outputs[i]}, output = {ret}")
        all_passed = False

"All tests passed!" if all_passed else ""       

'All tests passed!'